In [55]:
import requests
import requests
from bs4 import BeautifulSoup
import json

In [56]:
url = "http://www.pixiv.net/ranking.php"
response = requests.get(url)
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36', 
          'password': 'rd.)T6vF',
          'pixiv_id' : 'flicfloclol@hotmail.fr'}
response = requests.get(url, headers=headers, timeout = 10)

response.status_code

soup = BeautifulSoup(response.text)
links = []

for link in soup.find_all('a', class_="work _work"):
    links.append(link.get('href').split('/')[2])
    
for link in soup.find_all('a', class_="work _work multiple"):
    links.append(link.get('href').split('/')[2])
    
print(links)



['89109614', '89121548', '89095933', '89113282', '89095580', '89107668', '89116074', '89110432', '89107270', '89130385', '89095733', '89095813', '89095540', '89095717', '89096137', '89101314', '89101014', '89103040', '89099805', '89115885', '89115829', '89116623', '89111502', '89101234', '89110919', '89104336', '89117085', '89117161', '89102712', '89115849', '89115879', '89115888', '89127516', '89102857', '89095581', '89133180', '89101579', '89115904', '89115945', '89115906', '89100095', '89108877', '89096670', '89098204']


In [71]:
#url = "https://www.pixiv.net/ajax/user/103410/illusts?ids%5B%5D=" + links[0]
url = "https://www.pixiv.net/ajax/illust/89101579?lang=en"
response = requests.get(url)
response = requests.get(url, headers=headers, timeout = 7000)



print(response.status_code)

soup = BeautifulSoup(response.text)
#print(soup.get_text())

x = json.loads(response.text)
#print(x)

#parsed_json = json.loads(x)
#print(x['body']['tags']['tags'][0]['translation']['en'])
print(x['body']['tags']['tags'])

200
[{'tag': 'がんばれ同期ちゃん', 'locked': True, 'deletable': False, 'userId': '6210796', 'romaji': 'gannbaredoukichann', 'translation': {'en': 'Ganbare, Douki-chan'}, 'userName': 'よむ'}, {'tag': 'たこ焼き', 'locked': False, 'deletable': False, 'romaji': 'takoyaki'}, {'tag': 'オリジナル10000users入り', 'locked': False, 'deletable': False, 'romaji': 'orijinaruichimannyu-za-zuiri', 'translation': {'en': 'original 10000+ bookmarks'}}]
